# Room for the river: Gorssel 

## Imports 

In [1]:
#imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import sys

from ema_workbench import (Model, CategoricalParameter,ScalarOutcome, IntegerParameter, RealParameter,
                           MultiprocessingEvaluator, ema_logging, Constant, Policy, Scenario,
                           perform_experiments, SequentialEvaluator,Constraint)
from problem_formulation import get_model_for_problem_formulation

from dike_model_function import DikeNetwork  # @UnresolvedImport

from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress,GenerationalBorg)
from ema_workbench.em_framework.evaluators import (perform_experiments,BaseEvaluator)
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.analysis import plotting, plotting_util, parcoords, feature_scoring, prim
from ema_workbench import load_results 

from mpl_toolkits.mplot3d import Axes3D 

from sklearn import preprocessing

import time
import copy

import itertools

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import os
import functools

import random


In [2]:
from scipy.spatial.distance import pdist, squareform

def evaluate_diversity_single(indices, distances, weight=0.5, distance='euclidean'):
    '''
    takes the outcomes and selected scenario set (decision variables), 
    returns a single 'diversity' value for the scenario set.
    outcomes : outcomes dictionary of the scenario ensemble
    decision vars : indices of the scenario set
    weight : weight given to the mean in the diversity metric. If 0, only minimum; if 1, only mean
    '''
    i, j = [e for e in zip(*itertools.combinations(indices, 2))]
    subset_distances = distances[i, j]
    minimum = np.min(subset_distances)
    mean = np.mean(subset_distances)
    diversity = (1-weight)*minimum + weight*mean
    
    return [diversity]


def find_maxdiverse_scenarios(distances, combinations):
    scores = []
    for indices in combinations:
        diversity = evaluate_diversity_single(indices, distances)
        scores.append((diversity, indices))

    return scores

def optimize(scenario, nfe, model, converge_metrics, epsilons):
    with SequentialEvaluator(model) as evaluator:
    #with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=epsilons,
                                     reference=scenario)
    return results, convergence

def entries_to_remove(entries, the_dict):
    for key in entries:
        if key in the_dict:
            del the_dict[key]

In [3]:
# import the correct dataframes of a long run 
df_experiments_s1000_p10 = pd.read_csv("df_experiments_s1000_p10.csv")
df_outcomes_s1000_p10 = pd.read_csv("df_outcomes_s1000_p10.csv")

## Set experiments and outcomes 

In [4]:
print(df_experiments_s1000_p10.shape) 
print(df_outcomes_s1000_p10.shape)

(10000, 54)
(10000, 18)


In [5]:
#Run MORDM with the long run as imported, copy the data here so it doesn't have to be loaded in again 

experiments = df_experiments_s1000_p10.copy()
outcomes = df_outcomes_s1000_p10.copy()


# Scenario Selection 

### Choose outcomes of interest 

In [6]:
#random selecteren van outcomes 
set_seed = 1234567 
random.seed(set_seed)

In [7]:
#outcomes[['A.4_Expected Annual Damage','A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths','RfR Total Costs','Expected Evacuation Costs']]
number_of_outcomes = 173 #too much: 10000,1000,500,175,174 goes well: 150, 172,173

outcomes_of_interest = outcomes.sample(n=number_of_outcomes, random_state = set_seed)

y= (outcomes.index.isin(outcomes_of_interest.index))

#y = (outcomes['A.4_Expected Annual Damage']< max_expected_annual_damage)&( outcomes['Expected Evacuation Costs']<max_evacuation_costs)
#y

In [8]:
experiments_of_interest = experiments.loc[y]
outcomes_df = pd.DataFrame({k:v[y] for k,v in outcomes.items()})

#normalize outcomes on unit interval to ensure equal weighting of outcomes
x = outcomes_df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_outcomes = pd.DataFrame(x_scaled, columns=outcomes_df.columns)

In [9]:
outcomes_of_interest.head()
experiments_of_interest.head()

,Unnamed: 0,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
106,106,31,321.087153,10.0,0.339356,94.394245,1.5,0.251813,194.392592,10.0,...,9,3,7,6,9,10,2,116,0,dikesnet
336,336,78,58.368824,1.5,0.585965,112.850913,10.0,0.609583,109.966911,1.0,...,9,3,7,6,9,10,2,346,0,dikesnet
401,401,44,40.567925,1.0,0.706251,233.216342,1.0,0.140394,276.830567,1.5,...,9,3,7,6,9,10,2,411,0,dikesnet
490,490,116,221.408402,1.0,0.881096,96.943702,1.0,0.481245,238.827274,1.0,...,9,3,7,6,9,10,2,500,0,dikesnet
707,707,66,349.252960,1.5,0.565601,54.489702,10.0,0.399284,266.496110,10.0,...,9,3,7,6,9,10,2,717,0,dikesnet


### Create Combinations 

In [ ]:
# foutmelding: 
#The kernel for Documents/GitHub/EPA1361_group16/MORDM.ipynb appears to have died. It will restart automatically. 

n_scen = experiments.loc[y].shape[0]
indices = range(n_scen)
set_size = 5

n_scen
combinations = itertools.combinations(indices, set_size)
combinations = list(combinations)

In [ ]:
#sampled_combinations = random.sample(combinations, 100) #In case there are too many combinations 

In [ ]:
distances = squareform(pdist(normalized_outcomes.values))

cores = os.cpu_count()
partial_function = functools.partial(find_maxdiverse_scenarios, distances)


#Difference ProcessPool en ThreadPool 
# https://superfastpython.com/threadpoolexecutor-vs-processpoolexecutor/#Comparison_of_ThreadPoolExecutor_vs_ProcessPoolExecutor
#with ProcessPoolExecutor(max_workers=cores) as executor:

with ThreadPoolExecutor(max_workers=cores) as executor:
    worker_data = np.array_split(combinations, cores)
    results = [e for e in executor.map(partial_function, worker_data)]
    results = list(itertools.chain.from_iterable(results))
    

In [ ]:
results.sort(key=lambda entry:entry[0], reverse=True)
most_diverse = results[0]
most_diverse

### create scenarios 

In [ ]:
selected = experiments.loc[most_diverse[1], ['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model']]
scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]

In [ ]:
# Show what the scenarios entail 

# scenarios

In [ ]:
keys_notscenario = ['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', 
               '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', 
               '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 
              'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 
              'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 
              'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2', 
              'EWS_DaysToThreat', 'scenario', 'policy','model']

In [ ]:
#create dictionary with only the uncertainties 
for i in range(0,5):
    entries_to_remove(keys_notscenario, scenarios[i]) 

scenario_1 = scenarios[1]
scenario_1

In [ ]:
scenarios

# Mordm Run 

### Hypervolume determination 

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

results = []
for scenario in scenarios:
    convergence_metrics = [HyperVolume(minimum=[0,]*len(dike_model.outcomes), 
                                       maximum=[1000,]*len(dike_model.outcomes)),
                           EpsilonProgress()]
    epsilons = [0.1,]*len(dike_model.outcomes)
    
    results.append(optimize(scenario, 100, dike_model, convergence_metrics, epsilons))

### Epsilon volume determination 

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results1 = evaluator.optimize(nfe=5e3, searchover='levers',
                                  epsilons=[1,]*len(dike_model.outcomes),reference = scenario_1)

## Plots 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
for i, (_, convergence) in enumerate(results):
    ax1.plot(convergence.nfe, convergence.hypervolume, label=f'scenario {i}')
    ax2.plot(convergence.nfe, convergence.epsilon_progress, label=f'scenario {i}')

ax1.set_ylabel('hypervolume')
ax1.set_xlabel('nfe')
ax2.set_ylabel('$\epsilon$ progress')
ax2.set_xlabel('nfe')
fig.legend()
plt.show()

# The part of the notebook below is not of interest for now 

In [ ]:
# colors = iter(sns.color_palette())

# data = results[0][0].iloc[:, 5::]
# limits = parcoords.get_limits(data)

# limits.loc[0, ['inertia', 'reliability']] = 1
# limits.loc[0, 'max_P'] = 4 # max over results based on quick inspection not shown here
# limits.loc[0, 'utility'] = 1 # max over results based on quick inspection not shown here
# limits.loc[1, :] = 0
# paraxes = parcoords.ParallelAxes(limits)


# for i, (result, _) in enumerate(results):
#     color = next(colors)
#     data = result.iloc[:, 5::]
#     paraxes.plot(data, label=f'scenario {i}', color=color)

# paraxes.legend()
# plt.show()



## Deep Uncertainty 

In [ ]:
# #specify the policies 

policies = []
for i, (result, _) in enumerate(results):
    result = result.iloc[:, 0:5]
    for j, row in result.iterrows():
        policy = Policy(f'scenario {i} option {j}', **row.to_dict())
        policies.append(policy)
        
# ## Our way: 

# #Identify policies based on open exploration 

# policies = [Policy('policy 1', **{'0_RfR 0':1,
#                                   '0_RfR 1':1,
#                                   '0_RfR 2':1,
#                                   'A.1_DikeIncrease 0':5,
#                                  'A.1_DikeIncrease 1':5}),
#            Policy('policy 2', **{'4_RfR 0':1,
#                                   '4_RfR 1':1,
#                                   '4_RfR 2':1,
#                                   'A.5_DikeIncrease 0':5}),
#            Policy('policy 3', **{'1_RfR 0':1,
#                                   '2_RfR 1':1,
#                                   '3_RfR 2':1,
#                                   'A.3_DikeIncrease 0':5})]
        

In [ ]:
with MultiprocessingEvaluator(model) as evaluator:
    reeevaluation_results = evaluator.perform_experiments(1000, policies=policies)

Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.



In [ ]:

experiments, outcomes = reeevaluation_results

#determine thresholds Based on table 4.3 from report 

max_expected_annual_damage = 9.28e+05
#max_dike_investement = 10000
max_number_deaths = 0.00009
#max_rfr_costs = 200000
#max_evacuation_costs = 5000

#as defined earlier: 
thresholds = {'A.4_Expected Number of Deaths':max_number_deaths, 
              'A.4_Expected Annual Damage':max_expected_annual_damage}
#, 'reliability':0.8}

overall_scores = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    scores = {}
    for k, v in outcomes.items():
        try:
            n = np.sum(v[logical]>=thresholds[k])
        except KeyError:
            continue
        scores[k] = n/1000 
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T

In [ ]:
limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(overall_scores)
plt.show()

Alternative way?? 

In [ ]:
# overall_scores = {}
# regret = []
# for scenario in experiments.scenario.unique():
#     logical = experiments.scenario==scenario
#     temp_results = {k:v[logical] for k,v in outcomes.items()}
#     temp_results = pd.DataFrame(temp_results)
#     temp_experiments = experiments[experiments.scenario==scenario]
        
#     best = temp_results.max()
#     best['max_P'] = temp_results.max_P.min()
#     scenario_regret = a - best
#     scenario_regret['policy'] = temp_experiments.policy.values    
#     regret.append(scenario_regret)



In [ ]:
# regret = pd.concat(regret)
# maxregret = regret.groupby('policy').max()

In [ ]:
# limits = parcoords.get_limits(maxregret)
# paraxes = parcoords.ParallelAxes(maxregret)
# paraxes.plot(maxregret)
# plt.show()